[reference](https://github.com/udacity/deep-reinforcement-learning/blob/master/reinforce/REINFORCE.ipynb) <br>
[pytorch ](https://pytorch.org/docs/stable/distributions.html) <br>
[1st rank cartpole](https://github.com/udacity/deep-reinforcement-learning/blob/master/hill-climbing/Hill_Climbing.ipynb)

In [4]:
%reload_ext autoreload
%autoreload 2
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv
import torch
import random
import numpy as np
import math
from EXITrl.table_base import TableBase
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from gridworld_env_2d_state import GridworldEnv2DState
import gym

In [29]:
try: env.close()
except: pass
env = gym.make('CartPole-v1')

class REINFORCE(ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.initialize()
    
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        trajectory = []
        for i in range(1000):
            action, log_prob = self.policy(state)
            _state, reward, done, _ = self.env.step(action)
            trajectory.append((state, reward, done, log_prob))
            total_reward += reward
            state = _state
            if done: break
                
        loss = []
        for t in range(len(trajectory)):
            G = sum([self.gamma**k * reward for k, (state, reward, done, _) in enumerate(trajectory[t:])])
            _, _, _, log_prob = trajectory[t]
            # from Sutton's book said multiply by gamma**t 
            # but from David course no need too 
            # I tried both work
            loss.append(self.gamma**t * G * (-log_prob))
        self.update_policy(torch.stack(loss).sum())
        return total_reward
    
s = REINFORCE(env, 
               num_episodes=500,
               policy="softmax_policy",
               alpha=0.007, 
               gamma=.99)
s.train(True)


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0 reward: 12.0
episode: 1 reward: 17.0
episode: 2 reward: 16.0
episode: 3 reward: 25.0
episode: 4 reward: 21.0
episode: 5 reward: 14.0
episode: 6 reward: 11.0
episode: 7 reward: 31.0
episode: 8 reward: 77.0
episode: 9 reward: 16.0
episode: 10 reward: 10.0
episode: 11 reward: 25.0
episode: 12 reward: 14.0
episode: 13 reward: 31.0
episode: 14 reward: 18.0
episode: 15 reward: 11.0
episode: 16 reward: 35.0
episode: 17 reward: 21.0
episode: 18 reward: 25.0
episode: 19 reward: 13.0
episode: 20 reward: 20.0
episode: 21 reward: 30.0
episode: 22 reward: 16.0
episode: 23 reward: 13.0
episode: 24 reward: 11.0
episode: 25 reward: 20.0
episode: 26 reward: 46.0
episode: 27 reward: 26.0
episode: 28 reward: 17.0
episode: 29 reward: 21.0
episode: 30 reward: 16.0
episode: 31 reward: 59.0
episode: 32 reward: 12.0
episode: 33 reward: 11.0
episode: 34 reward: 27.0
episode: 35 reward: 30.0
episode: 36

episode: 313 reward: 500.0
episode: 314 reward: 117.0
episode: 315 reward: 307.0
episode: 316 reward: 105.0
episode: 317 reward: 131.0
episode: 318 reward: 286.0
episode: 319 reward: 387.0
episode: 320 reward: 194.0
episode: 321 reward: 323.0
episode: 322 reward: 290.0
episode: 323 reward: 467.0
episode: 324 reward: 500.0
episode: 325 reward: 281.0
episode: 326 reward: 218.0
episode: 327 reward: 500.0
episode: 328 reward: 128.0
episode: 329 reward: 161.0
episode: 330 reward: 123.0
episode: 331 reward: 105.0
episode: 332 reward: 102.0
episode: 333 reward: 229.0
episode: 334 reward: 239.0
episode: 335 reward: 331.0
episode: 336 reward: 141.0
episode: 337 reward: 369.0
episode: 338 reward: 294.0
episode: 339 reward: 166.0
episode: 340 reward: 214.0
episode: 341 reward: 202.0
episode: 342 reward: 299.0
episode: 343 reward: 104.0
episode: 344 reward: 237.0
episode: 345 reward: 186.0
episode: 346 reward: 310.0
episode: 347 reward: 462.0
episode: 348 reward: 438.0
episode: 349 reward: 500.0
e

In [6]:
env.close()

### REINFORCE with Baseline

In [27]:
try: env.close()
except: pass
env = gym.make('CartPole-v1')

class REINFORCE_with_baseline(ApproxVBase, ApproxPolicyBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        hidden = 64
        features = torch.nn.Linear(self.num_state, hidden)
        model_q = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        model_policy = torch.nn.Sequential(
            features,
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, self.num_action),
        ).to(self.device)
        ApproxVBase.initialize(self, learning_rate_name="beta", model=model_q)
        ApproxPolicyBase.initialize(self, learning_rate_name="alpha", model=model_policy)
    
    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        trajectory = []
        for i in range(1000):
            action, log_prob = self.policy(state)
            _state, reward, done, _ = self.env.step(action)
            trajectory.append((state, action, reward, done, log_prob))
            total_reward += reward
            state = _state
            if done: break
                
        td_targets = []
        qs = []
        loss_policy = []
        for t in range(len(trajectory)):
            G = sum([self.gamma**k * reward for k, (state, action, reward, done, log_prob) in enumerate(trajectory[t:])])
            state, action, reward, done, log_prob = trajectory[t]
            td_error = G - self.get_q(state)[action]
            td_targets.append(G)
            qs.append(self.get_q(state)[action])
            loss_policy.append(self.gamma**t * td_error * (-log_prob))
        self.update_q(torch.Tensor(td_targets), torch.stack(qs))
        self.update_policy(torch.stack(loss_policy).sum())
        return total_reward
    
s = REINFORCE_with_baseline(env, 
               num_episodes=500,
               policy="softmax_policy",
               alpha=0.007, 
               beta=0.001,
               gamma=.99)
s.train(True)


/Users/epinyoanun/miniconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
episode: 0 reward: 19.0
episode: 1 reward: 38.0
episode: 2 reward: 27.0
episode: 3 reward: 12.0
episode: 4 reward: 58.0
episode: 5 reward: 21.0
episode: 6 reward: 21.0
episode: 7 reward: 14.0
episode: 8 reward: 13.0
episode: 9 reward: 12.0
episode: 10 reward: 20.0
episode: 11 reward: 17.0
episode: 12 reward: 12.0
episode: 13 reward: 19.0
episode: 14 reward: 13.0
episode: 15 reward: 15.0
episode: 16 reward: 23.0
episode: 17 reward: 12.0
episode: 18 reward: 12.0
episode: 19 reward: 19.0
episode: 20 reward: 17.0
episode: 21 reward: 12.0
episode: 22 reward: 57.0
episode: 23 reward: 13.0
episode: 24 reward: 15.0
episode: 25 reward: 65.0
episode: 26 reward: 31.0
episode: 27 reward: 17.0
episode: 28 reward: 20.0
episode: 29 reward: 59.0
episode: 30 reward: 21.0
episode: 31 reward: 30.0
episode: 32 reward: 17.0
episode: 33 reward: 19.0
episode: 34 reward: 77.0
episode: 35 reward: 80.0
episode: 36

episode: 323 reward: 10.0
episode: 324 reward: 10.0
episode: 325 reward: 9.0
episode: 326 reward: 9.0
episode: 327 reward: 9.0
episode: 328 reward: 10.0
episode: 329 reward: 10.0
episode: 330 reward: 8.0
episode: 331 reward: 9.0
episode: 332 reward: 8.0
episode: 333 reward: 11.0
episode: 334 reward: 9.0
episode: 335 reward: 9.0
episode: 336 reward: 9.0
episode: 337 reward: 12.0
episode: 338 reward: 8.0
episode: 339 reward: 10.0
episode: 340 reward: 9.0
episode: 341 reward: 9.0
episode: 342 reward: 11.0
episode: 343 reward: 10.0
episode: 344 reward: 10.0
episode: 345 reward: 10.0
episode: 346 reward: 10.0
episode: 347 reward: 8.0
episode: 348 reward: 10.0
episode: 349 reward: 10.0
episode: 350 reward: 10.0
episode: 351 reward: 10.0
episode: 352 reward: 11.0
episode: 353 reward: 10.0
episode: 354 reward: 9.0
episode: 355 reward: 8.0
episode: 356 reward: 10.0
episode: 357 reward: 10.0
episode: 358 reward: 9.0
episode: 359 reward: 11.0
episode: 360 reward: 9.0
episode: 361 reward: 9.0
epis

### It's really bad for table base problem

In [14]:
env = GridworldEnv2DState()
s = REINFORCE(env, 
               num_episodes=100,
               policy="softmax_policy",
               epsilon=0.01, 
               alpha=0.008, 
               gamma=.9)
s.train(True)

episode: 0 reward: -6.0
episode: 1 reward: -90.0
episode: 2 reward: -28.0
episode: 3 reward: -9.0
episode: 4 reward: -12.0
episode: 5 reward: 0.0
episode: 6 reward: -23.0
episode: 7 reward: -9.0
episode: 8 reward: -29.0
episode: 9 reward: -10.0
episode: 10 reward: -80.0
episode: 11 reward: -26.0
episode: 12 reward: -23.0
episode: 13 reward: -8.0
episode: 14 reward: 0.0
episode: 15 reward: -3.0
episode: 16 reward: -26.0
episode: 17 reward: -6.0
episode: 18 reward: -5.0
episode: 19 reward: 0.0
episode: 20 reward: -6.0
episode: 21 reward: 0.0
episode: 22 reward: -12.0
episode: 23 reward: -18.0
episode: 24 reward: -22.0
episode: 25 reward: 0.0
episode: 26 reward: -17.0
episode: 27 reward: -12.0
episode: 28 reward: 0.0
episode: 29 reward: -13.0
episode: 30 reward: -6.0
episode: 31 reward: 0.0
episode: 32 reward: 0.0
episode: 33 reward: -4.0
episode: 34 reward: -5.0
episode: 35 reward: -4.0
episode: 36 reward: -2.0
episode: 37 reward: -2.0
episode: 38 reward: -7.0
episode: 39 reward: 0.0
epi